In [46]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from dfply import *

In [47]:
import psycopg2

conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password="June3060!!",
    database = "mimic_data",
    port="5432"
)

cursor = conn.cursor()
print("database connected")

database connected


In [48]:
df = pd.read_csv("../Data/reintubation_df_240127.csv", index_col="Unnamed: 0")
df2 = df.copy()
df2

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN


------

In [49]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_icu.d_items;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
d_items = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  0.06504988670349121


In [64]:
# itemid -> kg (224639, 226512), lbs (226531)

d_items[d_items["label"].str.contains("Weight", case=False)]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
766,224639,Daily Weight,Daily Weight,chartevents,General,kg,Numeric,NaN,NaN
1098,225124,Unintentional weight loss >10 lbs.,Unintentional weight loss >10 lbs.,chartevents,Adm History/FHPA,None,Checkbox,NaN,NaN
1856,226512,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,kg,Numeric,NaN,NaN
1866,226531,Admission Weight (lbs.),Admission Weight (lbs.),chartevents,General,None,Numeric,NaN,NaN
1950,226740,APACHE II Diagnosistic weight factors - Medical,Diagnostic weight category - Medical,chartevents,Scores - APACHE II,None,Text,NaN,NaN
1951,226741,APACHE II Diagnosistic weight factors - Surgic...,Diagnosistic weight categories - Surgical emer...,chartevents,Scores - APACHE II,None,Text,NaN,NaN
1952,226742,APACHE II Diagnosistic weight factors - Surgical,Diagnosistic weight categories - Surgical,chartevents,Scores - APACHE II,None,Text,NaN,NaN
1993,226846,Feeding Weight,Feeding Weight,chartevents,General,kg,Numeric,NaN,NaN
2485,227854,Weight Bearing Status,Weight Bearing Status,chartevents,OT Notes,None,Text,NaN,NaN


In [51]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_icu.chartevents where itemid in (226512, 226531, 224639);")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
weights_chart = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  17.74603033065796


In [52]:
weights_chart["itemid"].value_counts()

itemid
224639    202491
226531    119251
226512     67826
Name: count, dtype: int64

In [53]:
lbs_idx = weights_chart[weights_chart["itemid"] == 226531].index

In [54]:
# lbs 데이터를 kg으로 변환
# 1lbs -> 0.453592

weights_chart.loc[lbs_idx, "valuenum"] = round(weights_chart.loc[lbs_idx, "valuenum"] * 0.453592, 1)

In [55]:
weights_chart

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,88981,2180-07-23 12:36:00,2180-07-23 14:45:00,226512,39.4,39.4,kg,0
1,10000032,29079034,39553978,88981,2180-07-23 14:22:00,2180-07-23 14:23:00,226531,86.7,39.3,None,0
2,10000032,29079034,39553978,88981,2180-07-23 14:44:00,2180-07-23 14:45:00,226531,86.7,39.3,None,0
3,10000980,26913865,39765666,11933,2189-06-27 18:02:00,2189-06-27 18:02:00,226531,167.6,76.0,None,0
4,10000980,26913865,39765666,11933,2189-06-27 20:13:00,2189-06-27 20:13:00,226531,167.6,76.0,None,0
...,...,...,...,...,...,...,...,...,...,...,...
389563,19999828,25744818,36075953,44654,2149-01-08 16:45:00,2149-01-08 18:15:00,226512,67.9,67.9,kg,0
389564,19999828,25744818,36075953,59573,2149-01-10 00:00:00,2149-01-10 06:23:00,224639,67.7,67.7,kg,0
389565,19999840,21033226,38978960,1195,2164-09-13 02:00:00,2164-09-13 02:04:00,224639,78.4,78.4,kg,0
389566,19999840,21033226,38978960,82641,2164-09-12 09:26:00,2164-09-12 09:47:00,226512,77.5,77.5,kg,0


In [56]:
def group_df_weights(group):
    for idx, row in group.iterrows():
        if pd.notnull(row["int_stayid"]):
            
            weights_df = weights_chart[(weights_chart["subject_id"] == row["subject_id"]) & (weights_chart["hadm_id"] == row["hadm_id"]) & (weights_chart["stay_id"] == row["int_stayid"])]
            suitable_location = weights_df[weights_df["charttime"] <= row["intubationtime"]].last_valid_index()
            
            try:
                group.loc[idx, "weights"] = weights_df.at[suitable_location, "valuenum"]
            except:
                try:
                    group.loc[idx, "weights"] = weights_df.at[weights_df["charttime"].idxmin(), "valuenum"]
                except:
                    group.loc[idx, "weights_marker"] = "suitable_location is null, about int_stayid"

        else:
            
            weights_df = weights_chart[(weights_chart["subject_id"] == row["subject_id"]) & (weights_chart["hadm_id"] == row["hadm_id"]) & (weights_chart["stay_id"] == row["ext_stayid"])]
            suitable_location = weights_df[weights_df["charttime"] >= row["extubationtime"]].first_valid_index()
            
            try:
                group.loc[idx, "weights"] = weights_df.at[suitable_location, "valuenum"]
            except:
                try:
                    group.loc[idx, "weights"] = weights_df.at[weights_df["charttime"].idxmax(), "valuenum"]
                except:
                    group.loc[idx, "weights_marker"] = "suitable_location is null, about ext_stayid"
                    
    return group

In [57]:
df_weights=df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_weights)

In [58]:
df_weights

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,weights,weights_marker
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN,65.0,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN,99.6,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,80.4,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0,75.8,NaN
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN,75.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0,58.9,NaN
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN,58.9,NaN
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN,55.7,NaN
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN,107.5,NaN


In [59]:
df_weights["weights"].isna().sum()

66

In [60]:
df_weights[df_weights["weights"].isna()]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,weights,weights_marker
251,10298813,28375400,37697909.0,2168-12-14 18:02:00,2168-12-20 22:00:00,224385.0,100.0,NaN,NaN,NaN,NaN,NaN,2168-12-21 06:55:00,2168-12-21 06:55:00,NaN,NaN,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
442,10495665,23675676,39279580.0,2158-09-29 17:59:00,2158-09-29 19:00:00,224385.0,88.0,36086196.0,2158-10-06 16:19:00,227194.0,88.0,Planned Extubation,2158-10-18 11:45:00,NaN,NaN,9919.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
602,10684286,27967192,39571320.0,2173-11-04 19:27:00,2173-11-21 22:30:00,224385.0,68.9,39571320.0,2173-11-22 12:53:00,227194.0,68.9,Planned Extubation,2173-11-22 13:15:00,2173-11-22 13:15:00,NaN,863.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
724,10806973,20436523,37277371.0,2134-10-25 17:27:00,2134-11-05 05:55:00,224385.0,106.0,37445602.0,2134-11-08 03:15:00,225468.0,106.0,Unplanned Extubation (patient-initiated),2134-11-13 15:20:00,NaN,NaN,4160.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
831,10910797,23479816,NaN,2142-01-05 22:40:00,NaN,NaN,NaN,37298868.0,2142-01-12 12:50:00,227194.0,88.6,Planned Extubation,2142-01-19 19:08:00,NaN,NaN,NaN,2142-01-13 23:45:00,2095.0,NaN,"suitable_location is null, about ext_stayid"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8348,19330004,26432998,NaN,2187-12-30 16:33:00,NaN,NaN,NaN,33297641.0,2188-01-15 17:57:00,227194.0,69.0,Planned Extubation,2188-01-21 13:40:00,NaN,NaN,NaN,NaN,NaN,NaN,"suitable_location is null, about ext_stayid"
8350,19332871,24506790,31026776.0,2186-01-12 03:34:00,2186-01-12 08:39:00,224385.0,128.6,31026776.0,2186-01-13 15:00:00,227194.0,128.6,Planned Extubation,2186-02-03 11:26:00,NaN,NaN,1821.0,2186-01-17 17:08:00,5888.0,NaN,"suitable_location is null, about int_stayid"
8361,19341576,28598953,32896970.0,2160-02-01 13:13:00,2160-02-04 11:25:00,224385.0,73.4,32896970.0,2160-02-04 15:08:00,227194.0,73.4,Planned Extubation,2160-02-07 17:30:00,NaN,NaN,223.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
8819,19788246,20404156,35970013.0,2122-12-19 22:17:00,2122-12-23 01:55:00,224385.0,66.2,NaN,NaN,NaN,NaN,NaN,2122-12-23 09:40:00,2122-12-23 09:40:00,NaN,NaN,NaN,NaN,NaN,"suitable_location is null, about int_stayid"


In [61]:
weights_chart[weights_chart["subject_id"] == 10298813]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
10130,10298813,28375400,32401147,10571,2168-12-16 15:09:00,2168-12-16 15:09:00,226531,220,99.8,None,0
10131,10298813,28375400,32401147,10571,2168-12-17 14:56:00,2168-12-17 14:56:00,226531,220,99.8,None,0
10132,10298813,28375400,32401147,19091,2168-12-17 08:00:00,2168-12-17 07:40:00,224639,102.2,102.2,kg,0
10133,10298813,28375400,32401147,97514,2168-12-16 13:55:00,2168-12-16 14:30:00,226512,100,100.0,kg,0


# 내용정리

* d_items 에서 weights 관련된 itemid 확인, chartevents에서 해당되는 데이터만 추출
* lbs를 kg으로 변환
* weights를 paired된 data에 병합

## 문의사항

* NaN 값을 대체하는 방법..?
    * 함수에서 기본적으로 subject_id, hadm_id, (int_stayid or ext_stayid) 값이 동일한 값만 병합이 되었습니다.
    * 이 조건 때문에 nan값이 발생하는 것 같은데, 키에 비해서 체중은 가변성이 비교적 높기 때문에 다른 데이터를 좀 더 찾아봐야하는지 고민입니다.